# Control of a holonomic robot using Jupyter Notebook

## Overall description of the package:

The task of this Jupyter Notebook is to replace the User Interface of the [first assignment](https://github.com/jerin-joy/ResearchTrack2_Assignment1) in the Research Track 2 course. It involves the control of a holonomic robot by using Jupyter Notebook widgets and monitors the data obtained by using various graphical tools in matplotlib.

More specifically, the first part of this Jupyter Notebook interface is able of:
- Starting or stopping the robot "random position" behaviour by using two buttons - Start and Stop.
- Setting the linear and angular velocities of the robot by using two sliders.
- Directly controlling the robot movements by using 5 buttons (Forward, Turn right, Backward, Turn Left and Stop)

For the successful execution of this Jupyter notebook, the user have to parallelly run gazebo simulation in the same package by entering:

```
roslaunch rt2_assignment1 sim.launch
```
After successfully executed the Gazebo simulation, the user can run this Jupyter Notebook by selecting Kernel-> Restart & Run all.

# Headers

Headers that are used in the entire package are given below.

In [1]:
%matplotlib widget

from ipywidgets import Button, Layout, ButtonStyle, GridBox, Box, HBox, VBox
import ipywidgets as widgets

import numpy as np
import collections

import rospy
import actionlib
import actionlib.msg
import rt2_assignment1.msg
import jupyros as jr
from geometry_msgs.msg import Twist
from rt2_assignment1.srv import Command
from rosgraph_msgs.msg import Clock
from nav_msgs.msg import Odometry

import matplotlib.pyplot as plt
from matplotlib import animation, rc
import matplotlib.ticker as ticker


## Node initialization
Nodes used in ROS are initialized here. 

In [ ]:
ui_client = rospy.ServiceProxy('/user_interface', Command)

rospy.init_node('jupyter_interface')

action_service = actionlib.SimpleActionServer(
        '/go_to_point', rt2_assignment1.msg.MoveAction, auto_start=False)

action_client = actionlib.SimpleActionClient('/go_to_point', rt2_assignment1.msg.MoveAction)

global random_status


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


# Widgets

## Buttons

Buttons for directional control and starting and stopping the robot is mentioned below. To simulate the keyboard layout the buttons are placed in a gridbox. 

In [ ]:
#Buttons for directional control

up = Button(description='\u25b2', #direction='up',
                 layout=Layout(width='80%', grid_area='up'),
                 style=ButtonStyle(button_color='lightgray'))
down = Button(description='\u25bc', #direction='down',
                 layout=Layout(width='80%', grid_area='down'),
                 style=ButtonStyle(button_color='lightgray'))
right = Button(description='\u25b6', #direction='right',
                 layout=Layout(width='85%', grid_area='right'),
                 style=ButtonStyle(button_color='lightgray'))
left = Button(description='\u25c0', #direction='left',
                 layout=Layout(width='85%', grid_area='left'),
                 style=ButtonStyle(button_color='lightgray'))
stop_key = Button(description='STOP', #Stop button
                 layout=Layout(width='80%', grid_area='stop_key'),
                 style=ButtonStyle(button_color='orange'))

gb = GridBox(children=[up, down, right, left, stop_key],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto',
            grid_template_columns='33% 33% 33%',
            grid_template_areas='''
            ". up ."
            "left down right "
            ". stop_key ."
            ''')
       )

# display(gb)

#Buttons for starting and stopping of the robot.

start  = Button(description='START',
                 layout=Layout(width='80%', grid_area='start'),
                 style=ButtonStyle(button_color='lightgreen'))
stop  = Button(description='STOP',
                 layout=Layout(width='80%', grid_area='stop'),
                 style=ButtonStyle(button_color='lightblue'))

# h1 = VBox([start, stop])

g_all = GridBox(children=[start, stop],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto',
            grid_template_columns='31%',
            grid_template_areas='''
            ". start ."
            ". stop ."
            ''')
       )

# display(g_all)


## Sliders

Sliders that are used to control the robot by setting the linear and angular velocity of the robot are implemented here. When we increase the values in the slider, it gets published in the cmd_vel topic. 

In [ ]:
lin_slider = widgets.FloatSlider(min=0.0, max=1.0, description='Lin. Velocity')
# display(lin_slider)

ang_slider = widgets.FloatSlider(min=0.0, max=1.0, description='Ang. Velocity')
# display(ang_slider)

# Control of the robot using widgets

This part of the package explains about the control of the robot using various Jupyter Notebook widgets such as buttons and sliders. 

## Start/Stop the robot using buttons
Using start and stop buttons, the Jupyter Notebook is able to start and stop the movement of the robot in the gazebo environment. When the stop button is pressed, the movement of the robot is stopped by cancelling the goal of the robot.

In [ ]:
ui_client("stop")
random_status = False
output = widgets.Output()

def start_button(start):
    global random_status
    with output:
        slider_lin = 1.0
        slider_ang = 1.0
        rospy.set_param("slider_lin_scale",slider_lin)
        rospy.set_param("slider_ang_scale",slider_ang)
        ui_client("start")
        if random_status == True:
            print("Robot is already moving")
        random_status = True
        print('Start clicked')
        
def stop_button(stop):
    global random_status
    with output:
        ui_client("stop")
        if random_status == False:
            print("Robot is already stopped")
        random_status = False        
        print('Stop clicked')
        
start.on_click(start_button)
stop.on_click(stop_button)


## Setting the linear and angular velocity of the robot
In this part, the linear and angular velocity of the robot can be controlled using two sliders, each meant for controlling the linear speed and angular speed of the robot. 

In [ ]:
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

msg = Twist()

def on_value_change(change):
    if random_status == False:
        msg.linear.x = change['new']
        pub.publish(msg)
    else:
        slider_lin = change['new']
        rospy.set_param("slider_lin_scale",slider_lin)
        print("Random position movement active, velocity modulated")
    
def on_valueang_change(change):
    if random_status == False:
        msg.angular.z = change['new']
        pub.publish(msg)
    else:
        slider_ang = change['new']
        rospy.set_param("slider_ang_scale",slider_ang)
        print("Random position movement active, angular velocity modulated")

lin_slider.observe(on_value_change, names='value')
ang_slider.observe(on_valueang_change, names='value')

## Controlling the robot using 5 buttons
Here, the robot is controlled by using 5 buttons (Forward, Turn right, Backward, Turn Left and Stop). The buttons are directly published on the /cmd_vel topic.

In [ ]:
def up_button(up):
    global random_status
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, click again to start")
        random_status = False
    elif random_status == False:
        msg.linear.x = 1.0
        msg.angular.z = 0.0
        pub.publish(msg)
        print("Go straight")  
            
def down_button(down):
    global random_status
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, click again to start")
        random_status = False
    elif random_status == False:
        msg.linear.x = -1.0
        msg.angular.z = 0.0
        pub.publish(msg)
        print("Go backwards")

def right_button(right):
    global random_status
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, click again to start")
        random_status = False
    elif random_status == False:
        msg.linear.x = 0.0
        msg.angular.z = 1.0
        pub.publish(msg)
        print("Turn right")
            
def left_button(left):
    global random_status
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, click again to start")
        random_status = False
    elif random_status == False:
        msg.linear.x = 0.0
        msg.angular.z = -1.0
        pub.publish(msg)
        print("Turn left")
            
def stop_button_click(stop_key):
    global random_status
    if random_status == True:
        ui_client("stop")
        print("Random go to point stopped, manual mode active.")
        random_status = False
    elif random_status == False:
        msg.linear.x = 0.0
        msg.angular.z = 0.0
        pub.publish(msg)
        print('Robot stopped')
        
up.on_click(up_button)
down.on_click(down_button)
right.on_click(right_button)
left.on_click(left_button)
stop_key.on_click(stop_button_click)

# display(gb)

## Widgets Display

Every widgets are displayed here. The widgets can be used to start/stop the robot by clicking Start/Stop button, change the linear/angular velocity by two sliders and control the robot using 5 buttons.   

### Start/Stop button

When the user press the start button, the robot moves to a random position. And when the stop button is pressed, the goal is cancelled and the robot stops instantly. And the user can start the robot again by pressing the start button and the robot starts moving to next random position.

In [ ]:
display(g_all)

### Change linear/angular velocity
The user can change the linear and angular velocity of the robot by moving the slider towards the right. 

In [ ]:
display(lin_slider)
display(ang_slider)

### Control using 5 buttons
The user can control the robot by using 5 buttons - up, down, right, left and stop.

In [ ]:
display(gb)

# Plots

This part of the jupyter notebook contains:
- A line plot for visualizing cmd_vel vs. actual velocity (for linear and angular velocity)
- A bar plot displaying the number of reached targets and cancelled targets.
- A hist plot showing the time required to reach targets.
- An xy graph showing the robot’s position and the orientation.


## Calculation

In [ ]:
# target time and success rate calculation
number_of_targets = {'Reached':0, 'Cancelled':0}
time_to_target = []

def targetCallback(msg):
    global number_of_targets
    
    if msg.result.result:
        number_of_targets['Reached']+=1   
        time_to_target.append(msg.header.stamp.secs - msg.status.goal_id.stamp.secs)
    else:
        number_of_targets['Cancelled']+=1


rospy.Subscriber('/go_to_point/result', rt2_assignment1.msg.MoveAction, targetCallback)

## Plot Display

### cmd_vel vs actual velocity

In [ ]:
# Velocity feedback of the robot
fig_lin_vel, ax_lin_vel = plt.subplots()
ax_lin_vel.set_xlim(0, 100)
ax_lin_vel.set_ylim((-1.1, 1.1))
plt.title("Velocity check linear velocity")
plt.xlabel("Time [s]")
plt.ylabel("linear Velocity")

fig_ang_vel, ax_ang_vel = plt.subplots()
ax_ang_vel.set_xlim(0, 100)
ax_ang_vel.set_ylim((-1.1, 1.1))
plt.title("Velocity check angular velocity")
plt.xlabel("Time [s]")
plt.ylabel("angular Velocity")

# Array initilaization to store the value

time_vect = collections.deque(maxlen=400)
vel_target = collections.deque(maxlen=400)
vel_current = collections.deque(maxlen=400)
angular_target = collections.deque(maxlen=400)
angular_current = collections.deque(maxlen=400)

# linear velocity(v)
line_vel_target, = ax_lin_vel.plot([], [], '-b', markersize='5')
line_vel_current, = ax_lin_vel.plot([], [], '-r', markersize='5')
# angular velocity(w)
line_ang_target, = ax_ang_vel.plot([], [], 'xc', markersize='5')
line_ang_current, = ax_ang_vel.plot([], [], 'xg', markersize='5')


def init_vel():
   
    global line_vel_target, line_vel_current, line_ang_target, line_ang_current, ax_lin_vel, ax_ang_vel
    
    ax_lin_vel.legend()
    ax_lin_vel.set_title("Odom vs cmd_vel linear velocity")
    ax_lin_vel.set(xlabel='Time', ylabel='Linear velocity')
    ax_lin_vel.set_xlabel('time')

    ax_ang_vel.legend()
    ax_ang_vel.set_title("Odom vs cmd_vel angular velocity")
    ax_ang_vel.set(xlabel='Time', ylabel='Angular velocity')
    ax_ang_vel.set_xlabel('time')
    
     
    
    line_vel_target.set_data([],[])
    line_vel_current.set_data([],[])
    line_ang_current.set_data([],[])
    line_ang_target.set_data([],[])

    return (  line_vel_target,    line_vel_current, line_ang_current, line_ang_target)

def animate_vel(i):   
    ax_lin_vel.legend([line_vel_target ,line_vel_current], ['Cmd. linear', 'Linear Vel.'], loc = 'lower right')
    ax_ang_vel.legend([line_ang_target ,line_ang_current], ['Cmd. angular', 'ang Vel.'], loc = 'lower right')
    
    
    # linear velocity lines
    line_vel_target.set_data(time_vect, vel_target)   
    line_vel_current.set_data(time_vect, vel_current)  
    # angular velocity lines
    line_ang_target.set_data(time_vect, angular_target) 
    line_ang_current.set_data(time_vect, angular_current) 
    
    return (line_vel_target, line_ang_target, line_vel_current, line_ang_current)

def vel_callback(msg_cmd):   
    time_vect.append(rospy.get_time())       
    vel_target.append(msg_cmd.linear.x)  
    angular_target.append(msg_cmd.angular.z) 
    x_min = max([0, max(time_vect) - 20]) 
    x_max = max([100, max(time_vect)])
    ax_lin_vel.set_xlim( x_min, x_max )
    ax_ang_vel.set_xlim( x_min, x_max )

def odom_speed_callback(msg_odom):       
    vel_current.append(msg_odom.twist.twist.linear.x) 
    angular_current.append(-1.85*msg_odom.twist.twist.angular.z)

#robot_velocity data reading
jr.subscribe('cmd_vel', Twist, vel_callback)
#robot_odometery data reading
jr.subscribe('odom', Odometry, odom_speed_callback)

# Final output visual of odom and cmd_vel data
anim_l_velocity = animation.FuncAnimation(fig_lin_vel, animate_vel, frames = 1, init_func=init_vel, blit=True)

anim_a_velocity = animation.FuncAnimation(fig_ang_vel, animate_vel, frames = 1, init_func=init_vel, blit=True)

### Number of reached targets and cancelled targets

In [ ]:
bar_plot, fig = plt.subplots()
fig.set_ylabel("Number of Targets")
fig.set_title("Reached vs Cancelled Targets")
fig.set_ylim([0, max(number_of_targets.values())+1])


plot_value = plt.bar(list(number_of_targets.keys()), list(number_of_targets.values()), align='center')

def animate_plot(i):
    for plots, val in zip(plot_value, list(number_of_targets.values())):
         plots.set_height(val)   
    fig.set_ylim([0, max(list(number_of_targets.values()))+1]) 
    return (plot_value,) 

animate_targets = animation.FuncAnimation(bar_plot, animate_plot, frames=50, interval=20, blit=True) 

### Time required to reach targets

In [ ]:
fig_time, ax_time = plt.subplots()

ax_time.set_title("go_to_point time to reach goal")

ax_time.set_xlabel("time")
ax_time.set_ylabel("goals")
ax_time.set_ylim((0, 5))
ax_time.set_xlim((0, 40))

n_bins = 10;
time_plt, _, _ = ax_time.hist([], bins=n_bins, align='mid')

def animate_time(i):
    ax_time.cla()
    ax_time.set_title("go_to_point time to reach goal")

    ax_time.set_xlabel("time")
    ax_time.set_ylabel("goals")
    ax_time.set_ylim((0, 5))
    ax_time.set_xlim((0, 40))
    time_plt, _, _ = ax_time.hist(time_to_target, bins=n_bins, align='mid')
    return (time_plt,)

anim_time = animation.FuncAnimation(fig_time, animate_time, frames=50, interval=20, blit=True)

### Robot's position vs orientation

In [ ]:
# Robot current position 
fig, ax = plt.subplots()
ax.set_xlim(( -20, 20))
ax.set_ylim((-20, 20))
ax.set_aspect("equal")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.title("Robot current positon")

line, = ax.plot([], [], 'rx', markersize="5")

x_data =  collections.deque(maxlen=300)
y_data = collections.deque(maxlen=300)
theta_data = collections.deque(maxlen=300)

def init():
    line.set_data([], [])
    return (line,)
    
def animate(i):
    line.set_data(x_data, y_data)
    
    return (line)

def odom_callback(msg):
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x) 
      
                  
# Read robot odometry
jr.subscribe('/odom', Odometry, odom_callback)


anim_position = animation.FuncAnimation(fig, animate, init_func=init, blit=True)